In [ ]:
pip install ipynbcompress

In [ ]:
import os
from ipynbcompress import compress
filename = '/content/deep-learning-project (4).ipynb'
out = '/content/compressed.ipynb'
os.stat(filename).st_size
compress(filename, output_filename=out, img_width=800, img_format='jpeg')
compress(filename)
os.stat(filename).st_size

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory

class PlantDiseaseClassifier:
    def __init__(self, train_path, valid_path, test_path, img_width=256, img_height=256, batch_size=32):
        self.train_path = train_path
        self.valid_path = valid_path
        self.test_path = test_path
        self.img_width = img_width
        self.img_height = img_height
        self.batch_size = batch_size
        self.labels = None
        self.train_generator = None
        self.validation_generator = None
        self.test_generator = None
        self.model = None

    def load_data(self):
        train_datagen = ImageDataGenerator(rescale=1./255)
        self.train_generator = train_datagen.flow_from_directory(self.train_path, target_size=(self.img_width, self.img_height), batch_size=self.batch_size, class_mode='binary', shuffle=True)

        validation_datagen = ImageDataGenerator(rescale=1./255)
        self.validation_generator = validation_datagen.flow_from_directory(self.valid_path, target_size=(self.img_width, self.img_height), batch_size=self.batch_size, class_mode='binary', shuffle=False)

        test_datagen = ImageDataGenerator(rescale=1./255)
        self.test_generator = test_datagen.flow_from_directory(self.test_path, target_size=(self.img_width, self.img_height), batch_size=self.batch_size, class_mode=None, shuffle=False)

        self.labels = self.train_generator.class_indices

    def create_model(self):
        model = keras.Sequential([
            keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(self.img_width, self.img_height, 3)),
            keras.layers.MaxPooling2D((2, 2)),
            keras.layers.Conv2D(64, (3, 3), activation='relu'),
            keras.layers.MaxPooling2D((2, 2)),
            keras.layers.Conv2D(128, (3, 3), activation='relu'),
            keras.layers.MaxPooling2D((2, 2)),
            keras.layers.Conv2D(256, (3, 3), activation='relu'),
            keras.layers.MaxPooling2D((2, 2)),
            keras.layers.Conv2D(512, (3, 3), activation='relu'),
            keras.layers.MaxPooling2D((2, 2)),
            keras.layers.Flatten(),
            keras.layers.Dense(1568,activation="relu"),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(512, activation='relu'),
            keras.layers.Dense(len(self.labels), activation='softmax')
        ])

        opt = keras.optimizers.Adam(learning_rate=0.0001)
        model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.summary()
        self.model = model

    def train_model(self, epochs=20):
        history = self.model.fit(self.train_generator, validation_data=self.validation_generator,epochs=epochs)
        return history

    def evaluate_model(self):
        test_loss, test_accuracy = self.model.evaluate(self.validation_generator, steps=len(self.validation_generator))
        print(f"Test Loss:     {test_loss}")
        print(f"Test Accuracy: {test_accuracy}")
    def predict(self):
        predictions = self.model.predict(self.test_generator)
        y_pred = np.argmax(predictions, axis=1)
        return y_pred
    def plot_confusion_matrix(self, y_true, y_pred):
        confusion = confusion_matrix(y_true, y_pred)
        sns.heatmap(confusion, annot=True, fmt='d', cmap='jet', xticklabels=self.labels.keys(), yticklabels=self.labels.keys())
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title('Confusion Matrix')
        plt.show()

    def print_classification_report(self, y_true, y_pred):
        print(classification_report(y_true, y_pred, target_names=self.labels.keys()))

# Define paths to your dataset directories
train_path = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/'
valid_path = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/'
test_path = '/kaggle/input/new-plant-diseases-dataset/test/'

classifier = PlantDiseaseClassifier(train_path=train_path, valid_path=valid_path, test_path=test_path)
classifier.load_data()
classifier.create_model()
history = classifier.train_model(epochs=20)
classifier.evaluate_model()
y_pred = classifier.predict()


In [ ]:

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
print(y_pred)
len(y_pred)
import matplotlib.pyplot as plt
import numpy as np

test_filenames = classifier.test_generator.filenames
num_images = len(test_filenames)
num_rows = int(np.ceil(num_images / 5))
num_cols = min(num_images, 5)
plt.figure(figsize=(15, 3 * num_rows))
for i in range(num_images):
    # Load and display the image
    img_path = os.path.join(classifier.test_path, test_filenames[i])
    img = plt.imread(img_path)
    plt.subplot(num_rows, num_cols, i + 1)
    plt.imshow(img)
    plt.axis('off')

    # Get the predicted label
    pred_label_idx = y_pred[i]
    predicted_label = list(classifier.labels.keys())[pred_label_idx]

    # Print the predicted label
    plt.title(f'Label: {os.path.basename(test_filenames[i])}\nPredicted: {predicted_label}')

plt.tight_layout()
plt.show()


In [ ]:
print("Classification Report for Test Dataset:")
print(classification_report(y_pred, range(len(y_pred)), labels=range(len(classifier.labels.keys())), target_names=classifier.labels.keys()))
sns.heatmap(confusion_matrix(range(len(y_pred)), y_pred), annot=True, fmt='d', cmap='jet', xticklabels=classifier.labels.keys(), yticklabels=classifier.labels.keys())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix for Test Dataset')
plt.show()


In [ ]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Setting up paths
train_path = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/'
test_path = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/'

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_set = train_datagen.flow_from_directory(train_path,
                                              target_size=(224, 224),
                                              batch_size=32,
                                              class_mode='binary')

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size=(224, 224),
                                            batch_size=32,
                                            class_mode='binary')

# Loading VGG16 model
vgg = VGG16(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

# Freezing the pretrained layers
for layer in vgg.layers:
    layer.trainable = False

# Adding custom layers for classification
x = Flatten()(vgg.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(train_set.class_indices), activation='softmax')(x)

# Creating the final model
model = Model(inputs=vgg.input, outputs=predictions)

# Compiling the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Training the model
history = model.fit(train_set,
                    validation_data=test_set,
                    epochs=20)

# Plotting training and validation accuracy
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plotting training and validation loss
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19  # Modified import
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Setting up paths
train_path = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/'
test_path = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/'

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_set = train_datagen.flow_from_directory(train_path,
                                              target_size=(224, 224),  # Adjusted input shape
                                              batch_size=32,
                                              class_mode='binary')

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size=(224, 224),  # Adjusted input shape
                                            batch_size=32,
                                            class_mode='binary')

# Loading VGG19 model  # Modified
vgg = VGG19(input_shape=(224, 224, 3), weights='imagenet', include_top=False)  # Modified

# Freezing the pretrained layers
for layer in vgg.layers:
    layer.trainable = False

# Adding custom layers for classification
x = Flatten()(vgg.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(train_set.class_indices), activation='softmax')(x)

# Creating the final model
model = Model(inputs=vgg.input, outputs=predictions)

# Compiling the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Training the model
history = model.fit(train_set,
                    validation_data=test_set,
                    epochs=20)

# Plotting training and validation accuracy
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plotting training and validation loss
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()
